In [1]:
import time
import requests
from bs4 import BeautifulSoup
import json
import re
import time
import faker
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from dateutil import parser

In [2]:
# fake = faker.Faker(locale='ru')

In [3]:
# tinkoff_headers = {
#     'User-Agent': fake.chrome(),
#     'accept-language': 'en-US,en;q=0.9',
#     'pragma': 'no-cache',
#     'Host': 'cfg.tinkoff.org',
#     'Origin': 'https://www.tinkoff.ru',
#     'content-type': 'application/json',
#     'referer': 'https://www.tinkoff.ru'
# }

In [4]:
API_URL = "https://cfg.tinkoff.ru/vacancies/public/api/platform/v1/getVacanciesList"
VACANCY_API_URL = "https://cfg.tinkoff.ru/vacancies/public/api/platform/v2/getVacancy?urlSlug="

In [5]:
clear_string = lambda x: re.sub(' +', ' ', re.sub('<.*?>', ' ', x).replace('\n', '\n ')).strip()

In [6]:
def build_job_infos(cat='it'):
    search_payload = {
        'category': cat,
        'filters': {},
        'limit': 200
    }
    result = requests.post(API_URL, data=search_payload)
    job_infos = []
    for category in result.json()['response']['items']:
        for job_info in category['items']:
            title = job_info['title']
            url = VACANCY_API_URL + job_info['id']
            job_infos.append((title, url))
    return job_infos

In [7]:
total_jobs = build_job_infos('it') + build_job_infos('back-office')
len(total_jobs)

164

In [8]:
job_dicts = []
for title, url in tqdm(total_jobs):
    result = requests.get(url).json()['response']

    job_dict = {
        'title': title,
        'internal_id' : result['id'],
        'url': url,
        'description': clear_string(result['tasks']),
        'responsibilities': clear_string(result['responsibilities']),
        'qualifications': clear_string(result['requirements']),
        'company': 'Tinkoff',
        'publish_date': parser.parse(result['specialtyInfo']['createdAt'])
    }

    job_dicts.append(job_dict)

    time.sleep(0.2)  # 200 ms

100%|██████████| 164/164 [00:44<00:00,  3.65it/s]


In [10]:
snapshot = pd.DataFrame(job_dicts)
snapshot.sample(5)

,title,internal_id,url,description,responsibilities,qualifications,company,publish_date
49,Системный аналитик в Страхование,12c26269-1b7e-4f3c-a144-9d5861b03178,https://cfg.tinkoff.ru/vacancies/public/api/pl...,Миссия Тинькофф - &nbsp;произвести революцию н...,Сбор требований к системе в рамках проектов ил...,Опыт участия в развитии информационных систем ...,Tinkoff,2021-07-23 13:56:43.353000+00:00
121,Специалист сопровождения состоятельных клиентов,e1938679-4520-4219-b1bb-4827f4594e7b,https://cfg.tinkoff.ru/vacancies/public/api/pl...,Тинькофф Инвестиции стремительно расширяется и...,"Сопровождать VIP- клиентов по всем вопросам, с...",Знание основ рынка ценных бумаг Знание принцип...,Tinkoff,2020-01-22 13:06:23.488000+00:00
73,Руководитель направления аналитики (премиум),d9c86b2c-bfff-42b8-b161-b560535109e6,https://cfg.tinkoff.ru/vacancies/public/api/pl...,Тинькофф расширяет команду аналитики и отчетно...,Вести полный цикл управления командой аналитик...,"Уверенный пользователь SQL, Python Опыт управл...",Tinkoff,2018-03-27 09:05:15.018000+00:00
14,Business Data Engineer,8c08b803-47ee-4164-ad7b-da4fd121a13c,https://cfg.tinkoff.ru/vacancies/public/api/pl...,"У нас в DWH очень много данных: 6000 объектов,...",Быть экспертом по данным: помогать аналитикам ...,Высшее техническое образование Опыт работы с б...,Tinkoff,2018-03-27 09:04:23.643000+00:00
52,Системный аналитик (Tinkoff Asia),7b4e73fe-951c-4611-b4dd-01ec2ee3cb42,https://cfg.tinkoff.ru/vacancies/public/api/pl...,Тинькофф выходит на международный рынок. И Фил...,"Сбор и анализ бизнес требований, взаимодействи...",Опыт работы системным аналитиком от 3 лет Навы...,Tinkoff,2021-07-23 13:56:43.353000+00:00


In [14]:
snapshot['publish_date'] = snapshot['publish_date'].apply(lambda x: x.replace(tzinfo=None))
snapshot['publish_date'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 164 entries, 0 to 163
Series name: publish_date
Non-Null Count  Dtype         
--------------  -----         
164 non-null    datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 1.4 KB


In [15]:
snapshot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164 entries, 0 to 163
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   title             164 non-null    object        
 1   internal_id       164 non-null    object        
 2   url               164 non-null    object        
 3   description       164 non-null    object        
 4   responsibilities  164 non-null    object        
 5   qualifications    164 non-null    object        
 6   company           164 non-null    object        
 7   publish_date      164 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(7)
memory usage: 10.4+ KB


In [16]:
current_date = datetime.now().strftime('%d-%m-%Y')
current_date

'10-08-2022'

In [17]:
snapshot.to_csv(f'../data/tinkoff/{current_date}.csv')
snapshot.to_excel(f'../data/tinkoff/{current_date}.xlsx', engine='xlsxwriter')

In [135]:
requests.get(total_jobs[10][1]).json()['response']

{'id': '789e6068-504b-4278-baab-477a156fd814',
 'status': 'published',
 'title': 'PL/SQL разработчик',
 'specialty': 'Backend',
 'specialtyInfo': {'createdAt': '2018-03-27T09:04:23.643Z',
  'deletedAt': None,
  'id': '4c5c5e87-d961-4c74-a385-d0b25e36cdc5',
  'name': 'Backend',
  'order': 1,
  'urlSlug': 'back-end-razrabotka'},
 'city': 'Любой город',
 'experience': 'Middle',
 'tasks': '<p>Тинькофф — это огромная экосистема. Нашими клиентами уже стали 13 млн человек по всей России. Мы активно развиваемся, появляются новые продукты и сервисы, за которыми стоят разработчики и инженеры. Наши сервисы должны соответствовать высочайшим стандартам надежности и быстродействия. Поэтому сейчас мы ищем разработчика PL/SQL для усиления команд.</p>',
 'requirements': '<ul><li>Отличное знаешь PL/SQL (Oracle 10g и выше)</li><li>Понимаешь архитектуру oracle, основные структуры данных (индексы, таблицы), алгоритмы доступа к данным (index range/full/skip scan) и их соединения</li><li>Знаешь SQL и имеешь 